<a href="https://colab.research.google.com/github/shauryaraj5/Web-Scrapping-BBC-Weather-Data_py/blob/main/Proj_2_Web_Scrapping_BBC_Weather_Data_using_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import necessary Libraries

In [2]:
import json

from urllib.parse import urlencode

import requests
from bs4 import BeautifulSoup as bs

import pandas as pd
import re

from datetime import datetime

Get webpage from server

In [ ]:
required_city = "Mumbai"
location_url = 'https://locator-service.api.bbci.co.uk/locations?' + urlencode({
   'api_key' : 'AGbFAKx58hyjQScCXIYrxuEwJh2W2cmv',
   'stack' : 'aws',
   'locale' : 'en',
   'filter' : 'international',
   'place-types' : 'settlement,airport,district',
   'order':'importance',
   's' : required_city,
   'a' : 'true',
   'format' : 'json'
})
print(location_url)

In [4]:
result = requests.get(location_url).json()

In [5]:
url = 'https://www.bbc.com/weather/'+result['response']['results']['results'][0]['id']
webpage = requests.get(url)

We create an instance of beautifulsoup

In [ ]:
soup = bs(webpage.content, 'html.parser')
content = soup.prettify()
print(content)

We will be scraping all the high and low temp values with the summary.

In [ ]:
daily_ht = soup.find_all('span', class_='wr-day-temperature__high-value')
print(daily_ht)

In [ ]:
daily_lt = soup.find_all('span', attrs={'class' : 'wr-day-temperature__low-value'})
print(daily_lt)

In [ ]:
daily_sum = soup.find('div', class_='wr-day-summary')
print(daily_sum)

Now we will be post processing the data and extract relevant information like just the temperatures and we will remove all the html parts.

Creating a list of all High temp values

In [ ]:
daily_ht[0].text.strip()
daily_ht[0].text.strip().split()
daily_ht[0].text.strip().split()[0]

In [ ]:
daily_ht_list = [daily_ht[i].text.strip().split()[0] for i in range(len(daily_ht))]
print(daily_ht_list)

Creating list of all low temp values

In [ ]:
daily_lt_list = [daily_lt[i].text.strip().split()[0] for i in range(len(daily_lt))]
print(daily_lt_list)

Now we will be getting list of summaries.
since summary is just a long text with new descriptions starting with n upper case letter, we will use the upper case letter as the separator

In [ ]:
daily_sum.text

In [ ]:
res = ""
for i in daily_sum.text:
  if i.isupper():
    res += "*"+i
  else:
    res += i
daily_sum_list = res.split("*")
daily_sum_list.remove('')
print(daily_sum_list)

In [ ]:
datelist = pd.date_range(datetime.today(), periods = len(daily_ht)).tolist()
print(datelist)

In [ ]:
datelist = [datelist[i].date().strftime('%y-%m-%d') for i in range(len(datelist))]
print(datelist)

In [33]:
zipped = zip(datelist, daily_ht_list, daily_lt_list, daily_sum_list)


In [36]:
df = pd.DataFrame(list(zipped), columns=['Date', 'Highest', 'Lowest', 'Weather Summary'])

In [ ]:
display(df)

Save File as csv and xlsx

In [37]:
file_csv = required_city+'weather data'+'.csv'
df.to_csv(file_csv, index = None)

In [38]:
file_xlsx = required_city+'weather data'+'.xlsx'
df.to_excel(file_xlsx)